In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, Input, BatchNormalization, concatenate
from keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2
import tensorflow as tf
import imgaug as ia
import imgaug.augmenters as iaa
from albumentations import (
    RandomRotate90,Flip
)
import scipy.io
import difflib
from scipy import spatial
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import spatial

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print('This should list a GPU: ',tf.test.gpu_device_name())
os.chdir('/home/xx652230/Downloads/')

In [ ]:
num_features=64
modele = Sequential([
    Conv2D(filters=64, kernel_size=5, activation='relu', input_shape=(120, 120, 1)),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=16, kernel_size=5, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])
modele.load_weights("/home/xx652230/deap.hdf5")
modele.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
num_features = 64
width, height = 48, 48
modelf = Sequential([
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 2
        Dropout(0.5),

        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 6
        BatchNormalization(),
        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 7
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 8
        Dropout(0.5),

        Flatten(),

        Dense(2 * 2 * 2 * num_features, activation='relu'),  # 9
        Dropout(0.4),
        Dense(2 * 2 * num_features, activation='relu'),  # 10
        Dropout(0.4),
        Dense(2 * num_features, activation='relu'),  # 10
        Dropout(0.5),

        Dense(7, activation='softmax')

])
modelf.load_weights("./model/face/final_model_face.hdf5") 
modelf.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])



In [ ]:
extro=[[4.585985850208803e-15,0.21845765462726016,1.2004518046413182e-33, 0.0032726129400430133,2.88223566844931e-10],[4.617901587721846e-17, 2.8794632384214214e-05,0.7006698671272261,0.00030375642303191985,5.349446338041911e-29],[7.712894901000039e-26,6.793818882556078e-08,0.21594196027410054,7.898015419524533e-21,1.2083597473092602e-08],[2.029438337499281e-13,2.2545183913806833e-49,1.8604640313660267e-14,0.6551005018450482,6.454039690105326e-17],
[0.09947992749667031,1.8188108286179053e-07,3.254135222599444e-08,7.636538797314128e-07,2.0234142068112048e-36]]
probs=[[0.26,0.63,0.64,0.28,0.29],[0.25,0.64,0.67,0.39,0.25],[0.18, 0.57, 0.66, 0.36, 0.32],[0.19, 0.61, 0.49, 0.28, 0.22],[0.24, 0.58, 0.60, 0.17, 0.4]]
pb=np.array(extro)*np.array(probs)
ex=[]
for i in range(5):
    ex.append(np.sum(pb[i]))


def new_loss(y_pred,y_actual):
    cce = tf.keras.losses.CategoricalCrossentropy()
    l1=cce(y_pred,y_actual) 
    lall=[0.8*l1+0.2*ex[i] for i in range(5)] 
    lout=keras.backend.max(lall)
    return lout

In [ ]:
a0=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
b0=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)
combined0 = concatenate([a0.output, b0.output])
z0 = Dense(256, activation="relu")(combined0)
z0 = Dense(5, activation="softmax")(z0)
model0 = Model(inputs=[a0.input, b0.input], outputs=z0)
for i in range(len(model0.layers)-2):
    model0.layers[i].trainable=False
    
model0.compile(loss=new_loss, optimizer='adam',metrics=['accuracy'])
model0.load_weights("./results/cnnconcat_p30.hdf5") 
#model0.load_weights("./redo/cnnpersonality.hdf5") 




In [ ]:
w1=[i*0.30 for i in model0.layers[31].get_weights()] #eeg
model0.layers[31].set_weights(w1)
w2=[i*0.70 for i in model0.layers[32].get_weights()] #face
model0.layers[32].set_weights(w2)

In [ ]:
def read_csv(path,d1,d2):
    file_read=pd.read_csv(path)
    construct=[]
    #construct.append([i for i in file_read.columns])
    for i in range(len(file_read)):
        construct.append(file_read.loc[i,:])
    if d2!=0:    
        file1=np.reshape(np.array(construct)[39:],(len(construct)-39,d1,d2,1))  
        return file1
    else:
        file1=np.reshape(np.array(construct)[39:],(len(construct)-39,d1))
        return file1

# Operation 1: CNN concate variants

In [ ]:
#loading train and test data
imps=["eeg","face","quad"]
eeg=[]
face=[]
quad_out=[]
score_out=[]
model_out=[]
eeg1=[]
face1=[]
quad_out1=[]
score_out1=[]
model_out1=[]
for j in range(1,31):
    notin=[8,12,28]
    if j not in notin:
      if j<10: 
        for i in imps:
            if i=="eeg":
                eeg.append(read_csv("./SortedSamples/participants/"+str(i)+"P0"+str(j)+".csv",120,120)) 
                eeg1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",120,120))     

            elif i=="face":   
                face.append(read_csv("./SortedSamples/participants/"+str(i)+"P0"+str(j)+".csv",48,48))
                face1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",48,48))

            elif i=="quad":
                quad_out.append(read_csv("./SortedSamples/participants/"+str(i)+"P0"+str(j)+".csv",5,0))
                quad_out1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",5,0))

          
      elif  j>9: 
        for i in imps:
            if i=="eeg":
                eeg.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",120,120))    
                eeg1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",120,120))     

            elif i=="face":   
                face.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",48,48))
                face1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",48,48))

            elif i=="quad":
                quad_out.append(read_csv("./SortedSamples/participants/"+str(i)+"P"+str(j)+".csv",5,0))
                quad_out1.append(read_csv("./t/"+str(i)+"P0"+str(j)+".csv",5,0))


In [ ]:
num=[i for i range(len(eeg))]
np.random.shuffle(num)
t1=[]
    

In [ ]:
x1_train=np.concatenate([eeg[i] for i in t1])
x2_train=np.concatenate([face[i] for i in t1])
y_train=np.concatenate([quad_out[i] for i in t1])
x1_test=np.concatenate([eeg1[i] for i in t2])
x2_test=np.concatenate([face1[i] for i in t2])
y_test=np.concatenate([quad_out1[i] for i in t2])

In [ ]:
model0.fit([np.array(x1_train),np.array(x2_train)],np.array(y_train),validation_data=([np.array(x1_test),np.array(x2_test)],np.array(y_test)),batch_size=256,epochs=10)
emotion_list = ["Q1","Q2","Q3","Q4","Neutral"]
c1=model0.predict([np.array(x1_test),np.array(x2_test)])
c2=np.argmax(c1,axis=1)
y_pred_classes=keras.utils.to_categorical(np.array(c2))
conf_matrix = confusion_matrix(np.argmax(np.array(y_test), axis=1), np.argmax(np.array(y_pred_classes),axis=1), normalize = 'true')
df_conf = pd.DataFrame(conf_matrix, index = [i for i in emotion_list],
                  columns = [i for i in emotion_list])
plt.figure(figsize = (5,5))
sns.heatmap(df_conf, annot=True, cmap = 'Blues', fmt = '.1', cbar = False)
plt.plot()

In [ ]:
model0.save_weights("./redo/cnnpersonality.hdf5")

# Operation 2: CNN-LSTM variants

In [ ]:
#loading training set(group of 10 participants)
imps=["eeg","face","quad"]
eeg=[]
face=[]
quad_out=[]
score_out=[]
model_out=[]
for i in imps:
    for j in range(1,3):
       #if j!=2: 
        if i=="eeg":
            eeg.append(read_csv("./SortedSamples/first10/"+str(i)+str(j)+"0.csv",120,120))
        elif i=="face":   
            face.append(read_csv("./SortedSamples/first10/"+str(i)+str(j)+"0.csv",48,48))
        elif i=="quad":
            quad_out.append(read_csv("./SortedSamples/first10/"+str(i)+str(j)+"0.csv",5,0))
      


In [ ]:
#loading testing set(group of 10 participants)
imps=["eeg","face","quad"]
eeg1=[]
face1=[]
quad_out1=[]
score_out1=[]
model_out1=[]
for i in imps:
    for j in range(3,4):
       #if j!=2: 
        if i=="eeg":
            eeg1.append(read_csv("./redo/"+str(i)+str(j)+"0.csv",120,120))
        elif i=="face":   
            face1.append(read_csv("./redo/"+str(i)+str(j)+"0.csv",48,48))
        elif i=="quad":
            quad_out1.append(read_csv("./redo/"+str(i)+str(j)+"0.csv",5,0))
       


In [ ]:
from keras.layers import TimeDistributed,Reshape
a=Model(inputs=modele.inputs,outputs=modele.layers[-4].output)
video=Input(shape=(None,120,120,1))
video1=Input(shape=(None,48,48,1))
#b=Model(inputs=model_dude.inputs,outputs=model_dude.layers[-8].output)
mod=Sequential()
mod.add(video)
mod.add(TimeDistributed(a))
#mod.add(TimeDistributed(Flatten()))
mod.add(LSTM(128,return_sequences=True))
mod.add(TimeDistributed(Dense(128, activation="relu",name="eeg")))

b=Model(inputs=modelf.inputs,outputs=modelf.layers[-8].output)

#b=Model(inputs=model_dude.inputs,outputs=model_dude.layers[-8].output)
mod1=Sequential()
mod1.add(video1)
mod1.add(TimeDistributed(b))
#mod1.add(TimeDistributed(Flatten()))
mod1.add(LSTM(128,return_sequences=True))
mod1.add(TimeDistributed(Dense(128, activation="relu"),name="face"))
cont=concatenate([mod.output,mod1.output])
z=TimeDistributed(Dense(128, activation="relu"))(cont)
z =TimeDistributed(Dense(5,activation="softmax"))(z)
model_dope=Model(inputs=[mod.input,mod1.input],outputs=z)
for i in range(len(model_dope.layers)-8):
    model_dope.layers[i].trainable=False
model_dope.load_weights("./results/cnnlstmconcat_p30.hdf5")
model_dope.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])


In [ ]:
def _load_data(data, steps = 40):  
    docX, docY = [], []
    for i in range(0, int(data.shape[0]/steps-1)):
        docX.append(data[i*steps:(i+1)*steps,:])
        #docY.append(data[(i*steps+1):((i+1)*steps+1),:])
    return np.array(docX)

In [ ]:
xx1=_load_data(np.array(x1_train))
xx2=_load_data(np.array(x2_train))
yy1=_load_data(np.array(y_train))
xx4=_load_data(np.array(x1_test))
xx5=_load_data(np.array(x2_test))
yy6=_load_data(np.array(y_test))

In [ ]:
history = model_dope.fit([np.array(xx1)[:200],np.array(xx2)[:200]], np.array(yy1)[:200], validation_data=([np.array(xx4)[:50],np.array(xx5)[:50]], np.array(yy6)[:50]),epochs=10, verbose=1,steps_per_epoch=10)  


In [ ]:
w1=[i*0.3 for i in model_dope.layers[6].get_weights()] #eeg
model_dope.layers[6].set_weights(w1)
w2=[i*0.7 for i in model_dope.layers[7].get_weights()] #face
model_dope.layers[7].set_weights(w2)

In [ ]:
model_dope.evaluate([np.array(xx4),np.array(xx5)],np.array(yy6))

In [ ]:
emotion_list=["Q1","Q2","Q3","Q4","Neutral"]
ytest1=[]
ytest2=[]
c1=model_dope.predict([np.array(xx4),np.array(xx5)])
for i in range(len(c1)):
    for j in range(len(c1[i])):
            ytest1.append(yy6[i][j])
            ytest2.append(c1[i][j])

y_pred_classes=np.argmax(ytest2,axis=1)
y_pred_classes = keras.utils.to_categorical(y_pred_classes)
conf_matrix = confusion_matrix(np.argmax(np.array(ytest1), axis=1), np.argmax(np.array(y_pred_classes),axis=1), normalize = 'true')
df_conf = pd.DataFrame(conf_matrix, index = [i for i in emotion_list],
                  columns = [i for i in emotion_list])
df_conf

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(df_conf, annot=True, cmap = 'Blues', fmt = '.1', cbar = False)
plt.savefig("./redo/lstmbaseline1.jpg")
plt.plot()